# 基本分类：服饰图像分类

该教程训练一个神经网络模型对服饰图形进行分类，例如运动鞋和T恤等。如果你不懂所有细节，没有关系。这是一个完整Tensorflow程序的快节奏的概述，代码有详细的解释。

该教程使用 [tf.keras](https://www.tensorflow.org/guide/keras), 一个Tensorflow高层API构建和训练Tensorflow模型。

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## 导入Fashion MNIST数据集

该教程使用[Fashion MNIST](https://github.com/zalandoresearch/fashion-mnisyit)数据集，它包括10个类别，70000张灰度图片。每张图片28*28的分辨率，图片内容是服饰。

Fashion MNIST是经典数据集[MNIST](http://yann.lecun.com/exdb/mnist/)的一个替代或增强。MNIST常作为计算机视觉机器学习程序的Hello World数据集，其需要识别的相对较简单的10个手写数字。 

Fashion MNIST比MNIST挑战性强一些，但两个数据集都相对较小，用来验证算法正确工作，是测试和调试代码的好起点。

这里60000张图片用作训练，10000张图片用作对分类模型进行评价。Tensorflow中可以直接访问Fashion MNIST数据集（第一次运行可能需要从网络上下载数据集到本地）：

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

数据装载返回Numpy数组：

* `train_images` 和 `train_labels` 数组作为*训练集*—模型用来学习的数据.
* 模型使用*测试集*进行评价, 测试集为`test_images`和`test_labels` 数组.

图片是28x28 NumPy数组，每个像素值从0到255.*标签* 是整数数组，值从0到9，每个数字表示服饰所属的不同类别：

<table>
  <tr>
    <th>标签</th>
    <th>类别</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

每张图片有唯一的标签，由于*类别名*没有包括在数据集中，这里将它们保存起来，供后面使用： 

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## 观察数据

在训练模型之前，我们来观察一些数据集的格式。下面代码显示训练集中有60000张图片，每个图片28*28像素：

In [ ]:
print(train_images.shape)

同样的，在训练集中有60000个标签：

In [ ]:
print(train_labels.shape)

每个标签是一个0到9的标签ID：

In [ ]:
print(train_labels)

在测试集中有10000张图片，每张图片也是28*28像素的：

In [ ]:
print(test_images.shape)

测试集包括10000个图片标签：

In [ ]:
print(test_labels.shape)

## 数据预处理

在训练网络前，数据需要预处理。如果显示训练集中第一张图片，你将发现像素值位于0到255间：

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

将这些图片喂给网络前，将它们的像素值规范化到0和1间，这只需要将像素值除以255。训练集和测试集要以相同的方式进行预处理：

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

为了验证数据的格式正确和可以用来构建和训练网络，让我们显示训练集中前25张图片，并在每张图片下面显示类别名：

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])  # 不显示刻度
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[test_labels[i]])
plt.show()

## 构建模型

构建网络模型需要配置模型网络层，然后编译模型。

### 设置层

神经网络的基本构建块是网络层，网络层从喂给它们的数据中抽取表示，我们希望这些表示对于要处理的问题有意义。

大多数深度学习需要选择不同的网络层，大多数网络层，例如`tf.keras.layers.Dense`, 有需要在训练过程中学习的参数。

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)  # 没有使用softmax激活函数，后面的损失需要使用logits参数
])

网络中第一层, `tf.keras.layers.Flatten`, 将图片的格式从2维的28-28转换成1维的784。将这一层可以想象成将图片的每行像素一起水平排列。这一层没有参数需要学习，只对数据进行格式化处理。

当图片像素扁平化后，网络包含两个 `tf.keras.layers.Dense` 层。它们是密集连接层，或全连接层。第一个密集层由128个节点或神经元，第二个密集层由10个节点，并返回一个logit数组，每个节点包含一个分数，指示图片属于10个类别中的一个。

### 编译模型

在模型训练前，它需要进行一些设置，这些设置在模型编译步完成：

* *损失函数* —度量模型在训练中的精度。损失越小，精度越高，训练过程使得模型朝最小化这个函数值的方向调整。.
* *优化器* —基于看到的训练数据和损失函数值如何更新模型。.
* *指标* —用来监测训练和测试。下面使用的指标是*accuracy*, 即正确分类的图片比例。

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 训练模型

训练神经网络模型需要下列步骤：

1. 喂训练数据给模型。本例中训练数据为 `train_images` 和 `train_labels` 数组.
2. 模型学习从图片到标签的映射.
3. 利用模型对测试集进行预测，本例中测试集为`test_images` 数组.
4. 利用测试集的参考结果 `test_labels` 数组验证模型效果.

### Feed模型

调用 `model.fit`方法启动训练，利用训练数据fit模型:

In [ ]:
model.fit(train_images, train_labels, epochs=10)

在模型训练中，损失和精度会被显示，在训练数据中，模型最终获得大约0.91的精度。

### 评价模型

接下来，评价模型在测试集中的性能:

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

结果显示在测试集上的精度略小于在训练集上的精度。这两者之间的差距表示 *overfitting*. 当机器学习模型在以前未见到的新数据上表现较差时，过拟合发生。过拟合模型记住了训练数据中的噪音和细节，从而影响它在新数据上的性能。更多细节参考:
*   [过拟合展示]
*   [防止过拟合策略]

### 预测

利用训练的模型，你可以对图片进行预测。本例的模型是线性输出[logits]. 接一个softmax层将线性输出转换成概率，概率比较容易理解. 

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_images)

print(predictions.shape)

这里，模型对测试集中每张图片进行预测，让我们看看第一个预测结果：

In [ ]:
predictions[0]

预测结果是包含10个数的数组，它表示图片属于10个衣服类别的置信概率。可以看看哪个类别有最高的置信概率:

In [ ]:
np.argmax(predictions[0])

因此，图片最确信是 `class_names[9]`. 检查测试集标签可以发现预测是正确的:

In [ ]:
test_labels[0]

画一张图来看看图片属于10个类别的详细情况。

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

### 验证预测

利用训练的模型可以对图片进行预测。

看看第一张图片、预测和预测数组。正确预测标签是蓝色的，错误的是红色的。数字给出的是预测标签的百分比。

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

让我们对多个预测结果画图看看。注意模型可能会有预测出错的情形。

In [ ]:
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

## 使用训练模型

最后，使用训练的模型对一张图片进行预测。

In [ ]:
# 从测试集中获得一张图片
img = test_images[1]

print(img.shape)
print(test_labels[1])

`tf.keras` 模型对数据批或数据集中的数据一次性预测做了优化，相应地，即使是一张图片，也需要将它加入到列表中。

In [ ]:
# 将图片加入到batch中，batch中只有它
img = np.expand_dims(img, 0)

print(img.shape)

现在预测图片的标签：

In [ ]:
predictions_single = probability_model.predict(img)

print(predictions_single)

In [ ]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`keras.Model.predict` 方法返回列表的列表，批次中每张图片一个结果列表，看看批次中第一张图片的预测结果:

In [ ]:
np.argmax(predictions_single[0])

模型正确的预测了图片的标签。